## Project Proposal
### By: Benjamin Giese & Brady Cassada

#### Identify your dataset

Within our project there will be two datasets, one containing National Hockey League (NHL) team statistics and a second set that has attendance data by team and year. The team data includes each team's overall performance for a single year (wins, goals, penalty minutes, etc.). We plan to combine a subset of each of thes two data sets in order to explore relationships between team performance and attendance.

Team Data: We will be using the 'Teams.csv' from this dataset specifically. [Dataset]('https://www.kaggle.com/open-source-sports/professional-hockey-database#Teams.csv') 

Attendance Data: ESPN has public data of a Team's Home/Away/Overall attendance from 2000-Present day. This isn't already formatted as we will have to collect it all and clean it to our needs for analyzing. [One year of data]('http://www.espn.com/nhl/attendance/_/year/2019')

We'll have to join these two datasets together in some way, this will be done via year and team name. Knowing the 'Teams.csv' data only goes up to 2011, and the ESPN data starts 2000 we will have completed data from the years 2000-2011. 

#### Objectives for analyzing your dataset

Our goal is to find a relationship between and ideally predict attendance based on one or more of the following factors:
* Wins
* Average goals per game
* Average penalty minutes
    
Some additional thoughts:
* The team stats data set includes data for teams that are in leagues other than the NHL. We will only be focusing on NHL teams.
* The first 10 rows of this table look pretty lame and you'll notice that they are missing a lot of values. Data collection has obviously come a long way in the last 100 years so the more recent data that we will be looking at (2000-2011) is pretty much free of missing values.
* Due to a lockout, there was no 2004-2005 NHL season.

In [1]:
# First Ten Records of Datset

import pandas as pd
#import matplotlib.pyplot as plt

df = pd.read_csv('Teams.csv')

df.head(10)

,year,lgID,tmID,franchID,confID,divID,rank,playoff,G,W,...,GA,name,PIM,BenchMinor,PPG,PPC,SHA,PKG,PKC,SHF
0,1909,NHA,COB,BKN,NaN,NaN,4,NaN,12,4,...,104,Cobalt Silver Kings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1909,NHA,HAI,MTL,NaN,NaN,5,NaN,12,4,...,83,Haileybury Hockey Club,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1909,NHA,LES,TBS,NaN,NaN,7,NaN,12,2,...,100,Les Canadiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1909,NHA,MOS,MOS,NaN,NaN,6,NaN,12,3,...,95,Montreal Shamrocks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1909,NHA,MOW,MTW,NaN,NaN,1,NaN,12,11,...,41,Montreal Wanderers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1909,NHA,OT1,STE,NaN,NaN,2,NaN,12,9,...,66,Ottawa Senators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1909,NHA,REN,REN,NaN,NaN,3,NaN,12,8,...,54,Renfrew Creamery Kings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1910,NHA,MOC,MTL,NaN,NaN,2,NaN,16,8,...,62,Montreal Canadiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1910,NHA,MOW,MTW,NaN,NaN,4,NaN,16,7,...,88,Montreal Wanderers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1910,NHA,OT1,STE,NaN,NaN,1,NaN,16,13,...,69,Ottawa Senators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Team Member: Brady Cassada/Ben Giese